In [1]:
import numpy as np
from random import shuffle
from math import log, floor
import pandas as pd
import tensorflow as tf
import tensorboard as tb
from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.utils import *
from keras.layers.advanced_activations import *
from collections import Counter
from keras import *
from keras.engine.topology import *
from keras.optimizers import *
import keras
# import pandas as pd
import glob
from sklearn.semi_supervised import *
import pickle
from keras.applications import *
from keras.preprocessing.image import *
from keras.losses import mse, binary_crossentropy
import pandas as pd # data frame
import numpy as np # matrix math
from scipy.io import wavfile # reading the wavfile
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
import matplotlib.pyplot as plt # to view graphs
import wave
from math import log, floor
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
import numpy as np
import pandas as pd
from sklearn.decomposition import *
from sklearn.cluster import KMeans
import sys, os
import random,math
from tqdm import tqdm ##
from xgboost.sklearn import XGBClassifier
from sklearn.utils import shuffle # shuffling of data
from random import sample # random selection
from tqdm import tqdm # progress bar
# audio processing
from scipy import signal # audio processing
from scipy.fftpack import dct
import librosa # library for audio processing
# import xgboost as xgb
# import lightgbm as lgb
# import catboost as ctb
from keras.utils import *
from sklearn.ensemble import *
import pickle
# from bayes_opt import BayesianOptimization
from logHandler import Logger
from utils import readCSV, getPath, writePickle,readPickle
from keras.regularizers import l2
from keras.callbacks import History ,ModelCheckpoint, EarlyStopping

import resnet
from random_eraser import get_random_eraser
from mixup_generator import MixupGenerator

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
def explode(df, columns):
    idx = np.repeat(df.index, df[columns[0]].str.len())
    a = df.T.reindex_axis(columns).values
    concat = np.concatenate([np.concatenate(a[i]) for i in range(a.shape[0])])
    p = pd.DataFrame(concat.reshape(a.shape[0], -1).T, idx, columns)
    return pd.concat([df.drop(columns, axis=1), p], axis=1).reset_index(drop=True)

In [26]:
df = pd.read_csv('result/cotrain_ens.csv')
df.label = df.label.str.split(' ')
df

,fname,label
0,00063640.wav,"[Shatter, Keys_jangling, Scissors]"
1,0013a1db.wav,"[Flute, Trumpet, Meow]"
2,002bb878.wav,"[Bass_drum, Knock, Gunshot_or_gunfire]"
3,002d392d.wav,"[Bass_drum, Knock, Double_bass]"
4,00326aa9.wav,"[Oboe, Bass_drum, Clarinet]"
5,0038a046.wav,"[Bass_drum, Double_bass, Electric_piano]"
6,003995fa.wav,"[Squeak, Telephone, Clarinet]"
7,005ae625.wav,"[Acoustic_guitar, Gong, Cello]"
8,007759c4.wav,"[Clarinet, Saxophone, Telephone]"
9,008afd93.wav,"[Saxophone, Violin_or_fiddle, Trumpet]"


In [27]:
df1 = explode(df, ['label'])
df1['conf'] = random.random()
df1

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  app.launch_new_instance()


,fname,label,conf
0,00063640.wav,Shatter,0.327075
1,00063640.wav,Keys_jangling,0.327075
2,00063640.wav,Scissors,0.327075
3,0013a1db.wav,Flute,0.327075
4,0013a1db.wav,Trumpet,0.327075
5,0013a1db.wav,Meow,0.327075
6,002bb878.wav,Bass_drum,0.327075
7,002bb878.wav,Knock,0.327075
8,002bb878.wav,Gunshot_or_gunfire,0.327075
9,002d392d.wav,Bass_drum,0.327075


In [28]:
df2 = pd.read_csv('result/ens_ans.csv')
df2.label = df2.label.str.split(' ')
df2

,fname,label
0,00063640.wav,"[Computer_keyboard, Shatter, Keys_jangling]"
1,0013a1db.wav,"[Flute, Oboe, Violin_or_fiddle]"
2,002bb878.wav,"[Bass_drum, Knock, Computer_keyboard]"
3,002d392d.wav,"[Bass_drum, Knock, Snare_drum]"
4,00326aa9.wav,"[Oboe, Flute, Glockenspiel]"
5,0038a046.wav,"[Knock, Bass_drum, Electric_piano]"
6,003995fa.wav,"[Glockenspiel, Squeak, Meow]"
7,005ae625.wav,"[Acoustic_guitar, Electric_piano, Gong]"
8,007759c4.wav,"[Telephone, Clarinet, Acoustic_guitar]"
9,008afd93.wav,"[Saxophone, Clarinet, Flute]"


In [29]:
df2 = explode(df2, ['label'])
df2['conf'] = random.random()
df2

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  app.launch_new_instance()


,fname,label,conf
0,00063640.wav,Computer_keyboard,0.882225
1,00063640.wav,Shatter,0.882225
2,00063640.wav,Keys_jangling,0.882225
3,0013a1db.wav,Flute,0.882225
4,0013a1db.wav,Oboe,0.882225
5,0013a1db.wav,Violin_or_fiddle,0.882225
6,002bb878.wav,Bass_drum,0.882225
7,002bb878.wav,Knock,0.882225
8,002bb878.wav,Computer_keyboard,0.882225
9,002d392d.wav,Bass_drum,0.882225


In [39]:
kk=[5]
kk.extend([x for x in range(len(df_com.columns)-1)])
kk

[5, 0, 1, 2]

In [49]:
df_com[0].astype('float') + df_com[1].astype('float')

,0,1
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
6,NaN,NaN
7,NaN,NaN
8,NaN,NaN
9,NaN,NaN


In [44]:
df_com = pd.merge(df1,df2,how='outer',on=['fname','label'])
kk = ['fname','label']
kk.extend([x for x in range(len(df_com.columns)-2) ])
df_com.columns = [kk]
df_com['conf'] = (df_com[0] + df_com[1])
df_com

ValueError: Wrong number of items passed 2, placement implies 1

In [25]:
df_com.groupby(['fname','label']).sum()

conf
fname        label                     
00063640.wav Computer_keyboard        1
             Keys_jangling            1
             Scissors                 1
             Shatter                  1
0013a1db.wav Flute                    1
             Meow                     1
             Oboe                     1
             Trumpet                  1
             Violin_or_fiddle         1
002bb878.wav Bass_drum                1
             Computer_keyboard        1
             Gunshot_or_gunfire       1
             Knock                    1
002d392d.wav Bass_drum                1
             Double_bass              1
             Knock                    1
             Snare_drum               1
00326aa9.wav Bass_drum                1
             Clarinet                 1
             Flute                    1
             Glockenspiel             1
             Oboe                     1
0038a046.wav Bass_drum                1
             Double_bass              1
             Electric_piano           1
             Knock                    1
003995fa.wav Clarinet                 1
             Glockenspiel             1
             Meow                     1
             Squeak                   1
...                                 ...
ffd64be4.wav Snare_drum               1
             Squeak                   1
ffd6e84f.wav Bass_drum                1
             Double_bass              1
             Drawer_open_or_close     1
             Gunshot_or_gunfire       1
ffd9d282.wav Clarinet                 1
             Flute                    1
             Oboe                     1
             Trumpet                  1
ffe76321.wav Bass_drum                1
             Fart                     1
             Gunshot_or_gunfire       1
ffe8d42d.wav Acoustic_guitar          1
             Computer_keyboard        1
             Gunshot_or_gunfire       1
             Writing                  1
ffeba2cd.wav Cello                    1
             Harmonica                1
             Trumpet                  1
             Violin_or_fiddle         1
fff6d073.wav Bass_drum                1
             Double_bass              1
             Knock                    1
             Snare_drum               1
fff8d22d.wav Bark                     1
             Fart                     1
             Meow                     1
             Squeak                   1
             Violin_or_fiddle         1

[38701 rows x 1 columns]